## Installing a library to read .docx files

In [83]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


## Data cleaning for Employee Data

In [84]:
import pandas as pd
from docx import Document


def read_docx(file_path):
    doc = Document(file_path)
    data = []

    for table in doc.tables:
        # sets the first row as headers
        headers = [cell.text for cell in table.rows[0].cells]
        for row in table.rows[1:]:
            data.append([cell.text for cell in row.cells])

    return pd.DataFrame(data, columns=headers)


file_path = 'employee.docx'
employee = read_docx(file_path)
employee.head()

,Employee ID,First Name,Last Name,Contact Number,Gender
0,e00001,Christopher,Thomas,7123456790,Male
1,e00002,Isabella,Collins,9123456788,Female
2,e00003,James,Allen,3123456788,Male
3,e00004,Elizabeth,Morris,4123456790,Female
4,e00005,David,Clark,6123456790,Male


#### Changing all 'Male' to 'M' and 'Female' to 'F'

In [85]:
# change accrodingly
employee['Gender'] = employee['Gender'].replace({'Male': 'M', 'Female': 'F'})

#### Checking for missing/duplicate data

In [86]:
# checking for missing data 
missing = employee.isna().sum()
print(missing)

Employee ID       0
First Name        0
Last Name         0
Contact Number    0
Gender            0
dtype: int64


In [87]:
# checking for duplicates 
duplicates = employee.duplicated().sum()
print(f"Number of duplicates: {duplicates}")
employee.head()

Number of duplicates: 0


,Employee ID,First Name,Last Name,Contact Number,Gender
0,e00001,Christopher,Thomas,7123456790,M
1,e00002,Isabella,Collins,9123456788,F
2,e00003,James,Allen,3123456788,M
3,e00004,Elizabeth,Morris,4123456790,F
4,e00005,David,Clark,6123456790,M


## Data cleaning for Customer Data

In [88]:
## Customer.csv
import pandas as pd
customer = pd.read_csv('customer.csv')
customer.head()

,Customer ID,First Name,Last Name,Company Name,Contact Number
0,c0001,John,Doe,ABC Company,5123456789
1,c0002,Jane,Smith,XYZ Corporation,7123456789
2,c0003,Michael,Johnson,Acme Inc.,9123456787
3,c0004,Emily,Williams,Beta Technologies,3123456789
4,c0005,David,Brown,Gamma Enterprises,4123456789


In [89]:
# checking for missing data 
missing = customer.isna().sum()
print(missing)

Customer ID       0
First Name        0
Last Name         0
Company Name      0
Contact Number    0
dtype: int64


In [90]:
# checking for duplicates 
duplicates = customer.duplicated().sum()
print(f"Number of duplicates: {duplicates}")
customer.head()

Number of duplicates: 0


,Customer ID,First Name,Last Name,Company Name,Contact Number
0,c0001,John,Doe,ABC Company,5123456789
1,c0002,Jane,Smith,XYZ Corporation,7123456789
2,c0003,Michael,Johnson,Acme Inc.,9123456787
3,c0004,Emily,Williams,Beta Technologies,3123456789
4,c0005,David,Brown,Gamma Enterprises,4123456789


#### Customer Data has no duplicates and no missing values


## Data cleaning for Order Data

In [91]:
# Order CSV
import pandas as pd
order = pd.read_csv('order.csv')
order.head()

,Order ID,Customer ID,Employee ID,Required Model Type,Required Accuracy,Order Date,Completion Date,Model ID,Price
0,S231255,c0058,e00010,NN,60.31,2023/11/25,2023/12/05,m0008,676
1,S210837,c0032,e00019,kNN,68.01,2021/09/25,2021/12/07,m0034,125
2,S210316,c0032,e00004,LR,70.39,2021/10/12,2021/12/17,m0075,266
3,S221093,c0014,e00014,NN,61.75,2022/03/25,2022/10/05,m0023,399
4,S210253,c0016,e00016,SVM,56.60,2021/10/22,2021/12/06,m0056,236


### Make sure all orders are made before the completion date (date of model assignment)

In [92]:
# converting date columns to datetime format
order['Order Date'] = pd.to_datetime(order['Order Date'], format='%Y/%m/%d')
order['Completion Date'] = pd.to_datetime(order['Completion Date'], format='%Y/%m/%d')

# checking if there are any orders that have order dates before completion dates
order[order['Order Date'] > order['Completion Date']]

,Order ID,Customer ID,Employee ID,Required Model Type,Required Accuracy,Order Date,Completion Date,Model ID,Price


Looks like there are no such orders

In [93]:
order.head()

,Order ID,Customer ID,Employee ID,Required Model Type,Required Accuracy,Order Date,Completion Date,Model ID,Price
0,S231255,c0058,e00010,NN,60.31,2023-11-25,2023-12-05,m0008,676
1,S210837,c0032,e00019,kNN,68.01,2021-09-25,2021-12-07,m0034,125
2,S210316,c0032,e00004,LR,70.39,2021-10-12,2021-12-17,m0075,266
3,S221093,c0014,e00014,NN,61.75,2022-03-25,2022-10-05,m0023,399
4,S210253,c0016,e00016,SVM,56.60,2021-10-22,2021-12-06,m0056,236


### Check for missing/duplicate data

In [94]:
# checking for missing data 
missing = order.isna().sum()
print(missing)

Order ID               0
Customer ID            1
Employee ID            0
Required Model Type    0
Required Accuracy      0
Order Date             0
Completion Date        0
Model ID               0
Price                  0
dtype: int64


#### There is one row with missing customer ID, we will be dropping this row.

In [95]:
# making sure there is no missing values
order.dropna(axis=1, how='any', inplace=True)

In [96]:
# checking for missing data 
missing = order.isna().sum()
print(missing)

Order ID               0
Employee ID            0
Required Model Type    0
Required Accuracy      0
Order Date             0
Completion Date        0
Model ID               0
Price                  0
dtype: int64


In [97]:
# checking for duplicates 
duplicates = order.duplicated().sum()
print(f"Number of duplicates: {duplicates}")

Number of duplicates: 0


## Data cleaning for Modeling Data

In [98]:
dataset = pd.read_excel('modeling.xlsx', sheet_name='dataset')
dataset.head()

,Dataset ID,Dataset Name
0,ds001,Retail_Sales_Data
1,ds002,Customer_Churn_Analysis
2,ds003,Stock_Market_Prices
3,ds004,Employee_Satisfaction_Survey
4,ds005,Marketing_Campaign_Performance


In [99]:
dataset.columns

Index(['Dataset ID', ' Dataset Name'], dtype='object')

#### We will remove the blank spaces in the columns

In [100]:
dataset.columns = dataset.columns.str.strip()
dataset.columns

Index(['Dataset ID', 'Dataset Name'], dtype='object')

#### Check for missing/duplicate data

In [101]:
# checking for missing data 
missing = dataset.isna().sum()
print(missing)

Dataset ID      0
Dataset Name    0
dtype: int64


In [102]:
# checking for duplicates 
duplicates = dataset.duplicated().sum()
print(f"Number of duplicates: {duplicates}")
dataset.head()

Number of duplicates: 0


,Dataset ID,Dataset Name
0,ds001,Retail_Sales_Data
1,ds002,Customer_Churn_Analysis
2,ds003,Stock_Market_Prices
3,ds004,Employee_Satisfaction_Survey
4,ds005,Marketing_Campaign_Performance


In [103]:
model = pd.read_excel('modeling.xlsx', sheet_name='model')
model.head()

,Model ID,Model Code,Training Date,Accuracy,Dataset ID
0,m0001,RF,2020/06/25,93.5,ds001
1,m0002,NN,2020/03/12,88.2,ds002
2,m0003,SVM,2020/09/18,85.7,ds003
3,m0004,kNN,2020/07/19,91.4,ds004
4,m0005,DT,2020/02/05,86.9,ds005


#### Changing Dates to datetime format for SQL

In [104]:
model['Training Date'] = pd.to_datetime(model['Training Date'], format='%Y/%m/%d')
model.head()

,Model ID,Model Code,Training Date,Accuracy,Dataset ID
0,m0001,RF,2020-06-25,93.5,ds001
1,m0002,NN,2020-03-12,88.2,ds002
2,m0003,SVM,2020-09-18,85.7,ds003
3,m0004,kNN,2020-07-19,91.4,ds004
4,m0005,DT,2020-02-05,86.9,ds005


In [105]:
# checking for missing data 
missing = model.isna().sum()
print(missing)

Model ID         0
Model Code       0
Training Date    0
Accuracy         0
Dataset ID       0
dtype: int64


In [106]:
# checking for duplicates 
duplicates = model.duplicated().sum()
print(f"Number of duplicates: {duplicates}")

Number of duplicates: 0


In [107]:
modelType = pd.read_excel('modeling.xlsx', sheet_name='modeltype')
modelType.head()

,Model Code,Model Type
0,DT,Decision Tree
1,RF,Random Forest
2,LR,Linear Regression
3,NN,Neural Network
4,SVM,Support Vector Machine


In [108]:
modelType.columns

Index(['Model Code', ' Model Type'], dtype='object')

#### Remove blank spaces in column name


In [109]:
modelType.columns = modelType.columns.str.strip()
modelType.columns

Index(['Model Code', 'Model Type'], dtype='object')

In [110]:
# checking for missing data 
missing = modelType.isna().sum()
print(missing)

Model Code    0
Model Type    0
dtype: int64


In [111]:
# checking for duplicates 
duplicates = modelType.duplicated().sum()
print(f"Number of duplicates: {duplicates}")

Number of duplicates: 0


#### Exporting the data as csv file for OLTP bulk insert

In [112]:
# saving the data as csv file
employee.to_csv('updated_employee.csv', index=False)

customer.to_csv('updated_customer.csv', index=False)

order.to_csv('updated_order.csv', index=False)

dataset.to_csv('dataset.csv', index=False)

model.to_csv('model.csv', index=False)

modelType.to_csv('modelType.csv', index=False)